In [1]:
import tensorflow as tf
from tensorflow.keras import layers, Sequential

## 模型乐园
对于常用的网络模型，如 ResNet， VGG 等，不需要手动创建网络，可以直接从 keras.applications 子模块下一行代码即可创建并使用这些经典模型，同时还可以通过设置 weights 参数加载预训练的网络参数。

### 1. 加载模型

In [3]:
# 加载ImageNet预训练网络模型，并去掉最后一层
resnet = tf.keras.applications.ResNet50(weights = 'imagenet',
                                    include_top = False)
resnet.summary()
# 需要从github上下载此模型

94773248/94765736 [==============================] - 1450s 15us/step
Model: "resnet50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, None, None,  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, None, None, 3 0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, None, None, 6 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, None, None, 6 256         conv1_conv[0][0]                 
______________________

In [5]:
# 测试网络的输出
x = tf.random.normal([4, 224, 224, 3])
out = resnet(x)
out.shape

TensorShape([4, 7, 7, 2048])

### 2. 在模型基础上进行修改

In [7]:
# 新建池化层
global_average_layer = layers.GlobalAveragePooling2D()

# 利用上一层的输出作为本层的输入，测试输出
x = tf.random.normal([4, 7, 7, 2048])
out = global_average_layer(x)
out.shape

TensorShape([4, 2048])

In [8]:
# 新建全连接层
fc = layers.Dense(100)
x = tf.random.normal([4, 2048])
out = fc(x)
out.shape

TensorShape([4, 100])

In [9]:
# 利用Sequential包装成一个新的网络
mynet = Sequential([resnet, global_average_layer, fc])
mynet.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet50 (Model)             (None, None, None, 2048)  23587712  
_________________________________________________________________
global_average_pooling2d (Gl (None, 2048)              0         
_________________________________________________________________
dense (Dense)                (None, 100)               204900    
Total params: 23,792,612
Trainable params: 23,739,492
Non-trainable params: 53,120
_________________________________________________________________


通过设置 resnet.trainable = False 可以选择冻结 ResNet 部分的网络参数，只训练新建的网络层，从而快速、高效完成网络模型的训练。